In [1]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import sys
import argparse
import csv
import getpass
import json
import time
import types, copy, pprint
import logging 
import re
import shutil
import yaml
from datetime import datetime
for p in ['./src','../..']:
    if p not in sys.path:
        print(f"insert {p}")
        sys.path.insert(0, p)
        
import wandb
import glob
from matplotlib import pyplot as plt
from typing import Dict, List, Tuple
from scipy.sparse import csr_matrix
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from torchinfo import summary

torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=150, profile=None, sci_mode=None)
print(torch.__version__)

pd.options.display.width = 132
pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=150, profile=None, sci_mode=None)

os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Train.ipynb"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
 
timestamp = datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

print(' Timestamp: ','./tmp/sql_cohort'+'_'+timestamp+'.txt')
# logLevel = os.environ.get('LOG_LEVEL', 'INFO').upper()
# FORMAT = '%(asctime)s - %(levelname)s: - %(message)s'
# logging.basicConfig(level="INFO", format= FORMAT)

insert ./src
insert ../..
2.2.1+cu118
 Timestamp:  ./tmp/sql_cohort_2024_07_08_20:14:26.txt


In [3]:
import snnl.utils as utils
from KevinsRoutines.utils.utils_general  import  list_namespace
# from KevinsRoutines.utils.utils_wandb  import  init_wandb, wandb_log_metrics,wandb_watch

# from snnl.modelsev_base import Model
# from snnl.models.dev_dnn import DNN 
from snnl.models import Autoencoder


In [4]:
import sklearn.metrics as skm 
import scipy.stats as sps 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform, euclidean
import matplotlib.pyplot as plt
import seaborn as sb

# from sklearn.metrics import accuracy_score, f1_score, roc_curve, roc_auc_score, classification_report, precision_recall_fscore_support
# from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, roc_curve, RocCurveDisplay
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
current_device = utils.get_device()
print(f"Current device is : {current_device}")

current_device = utils.set_device(2)

# Setup

#### main(args)

In [ ]:
# if __name__ == "__main__":
input_args = f" --seed              1234 " \
             f" --runmode           baseline" \
             f" --configuration     ./hyperparameters/autoencoder_cellpainting-50.yaml"
             f" --gpu_id             2 "\
             f" --cpb                200" 
             # f" --runmode           snnl" \
             # f" --configuration     hyperparameters/dnn_mnist.json"
args = utils.parse_args(input_args.split())
args

with open(args.configuration) as f:
    args = types.SimpleNamespace(**yaml.safe_load(f), **(vars(args)))
args.batch_size = args.cellpainting_args['batch_size']
args.compounds_per_batch = args.cellpainting_args['compounds_per_batch']

utils.set_global_seed(args.seed)

In [ ]:
trn_file_sz = args.cellpainting_args['train_end'] - args.cellpainting_args['train_start']
val_file_sz = args.cellpainting_args['val_end']   - args.cellpainting_args['val_start']
tst_file_sz = args.cellpainting_args['test_end']  - args.cellpainting_args['test_start']
smp_sz = args.cellpainting_args['sample_size']
buf_sz = args.cellpainting_args['compounds_per_batch']
bth_sz = args.batch_size
recs_per_batch = smp_sz * bth_sz * buf_sz

for file_sz in [tst_file_sz]:
    bth_per_epoch = file_sz // recs_per_batch
    print(f" - Each mini-batch contains {recs_per_batch/smp_sz} compounds with {smp_sz} samples per each compound : total {recs_per_batch} rows")
    print(f" - Number of {recs_per_batch} row full size batches per epoch: {bth_per_epoch}")
    print(f" - Rows covered by {bth_per_epoch} full size batches ({recs_per_batch} rows) per epoch:  {(file_sz // recs_per_batch) * recs_per_batch}")
    print(f" - Last partial batch contains : {file_sz % recs_per_batch} rows")
    print() 

### Load Cell Painting Dataset

In [ ]:
# args.cellpainting_args['validation_path']
args.cellpainting_args['val_start'] = 0
args.cellpainting_args['val_end'] = 3600
# args.cellpainting_args['training_path'] = '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv'
# args.cellpainting_args['validation_path'] = '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv'
args.cellpainting_args['train_start'] = 24_000
args.cellpainting_args['train_end'] = 27_600
args.cellpainting_args

In [ ]:
data_loader = dict()

if args.dataset == 'cellpainting':
    print(f" load {args.dataset}")
    train_dataset = utils.CellpaintingDataset(type = 'train',    **args.cellpainting_args)
    data_loader['train'] = utils.InfiniteDataLoader(dataset=train_dataset, batch_size= args.batch_size,shuffle = False, num_workers = 0, collate_fn = utils.custom_collate_fn)
    val_dataset = utils.CellpaintingDataset(type = 'val',    **args.cellpainting_args)
    data_loader['val'] = utils.InfiniteDataLoader(dataset=val_dataset, batch_size= args.batch_size,shuffle = False, num_workers = 0, collate_fn = utils.custom_collate_fn)
    test_dataset = utils.CellpaintingDataset(type = 'test',    **args.cellpainting_args)
    data_loader['test'] = utils.InfiniteDataLoader(dataset=test_dataset, batch_size= args.batch_size,shuffle = False, num_workers = 0, collate_fn = utils.custom_collate_fn)

# for idx, batch in enumerate(test_loader):
#     print(f"{idx:4d}, {batch[0].shape}, {batch[1].shape},{batch[2].shape},{batch[3].shape},{batch[4].shape}, {batch[3][:3]}, {batch[3][-3:]}")
#     utils.display_cellpainting_batch(idx, batch)
#     if idx >= 0:
#         break

In [ ]:
# input_shape = 784
# code_units =  20
args.temperature   = 0.00
args.loss_factor   = 1.0
args.snnl_factor   = 0.0e+00
args.learning_rate = 0.0e-00    ## 0.001
args.temperatureLR = 0.0e-00    ## 1e-4
# temperatureLR = 1e-03     ## 1e-4
print(f"   Units             {args.units}")
print(f"   Activations       {args.activations}")
print(f"   Latent dim        {args.code_units}")
print(f"   loss_factor       {args.loss_factor}")
print(f"   snnl_factor       {args.snnl_factor}")
print(f"   temperature       {args.temperature}")
print(f"   learning_rate     {args.learning_rate}")
print(f"   temperatureLR:    {args.temperatureLR}")

In [ ]:
list_namespace(args)

# Define Autoencoder Model

In [ ]:
model = utils.define_autoencoder_model(args, embedding_layer = 4)
model.device
# if args.runmode.lower() == "baseline":
#         adam_weight_decay = 0,
#         use_scheduler = True,
#         use_snnl   = False,
#         snnl_factor= 0.0,
#         use_temp_scheduler = False
# elif args.runmode.lower() == "snnl":
#         mode="latent_code",
#         adam_weight_decay = 0,
#         use_snnl=True,
#         use_annealing= False,        
#         use_sum = False,
#         SGD_weight_decay= 0,
#         use_temp_scheduler = False
# model.device

In [ ]:
# display_model_summary(model)

# Load Model from saved checkpoint - Run data through model

In [ ]:
# ex_runmode = 'snnl'
ex_runmode = 'baseline'
# ex_date = "20240515"
ex_date = "20240605"
ex_title="snglOpt_50Ltnt"
# ex_date = "20240516"
# ex_title="snglOpt_100Ltnt"
# ex_date = "20240605"
# ex_title="snglOpt_150Ltnt"

# ex_runmode = 'baseline'
# ex_title="snglOpt_50Ltnt"
# ex_title="firstTry"
ex_cpb = args.compounds_per_batch
epochs = 50
ex_factor = 0
# ex_title="dropout04"
# ex_title="secondTry"
# title="autoencoder"
# title='Xentropy'
# model.name

In [ ]:
# filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{epochs:03d}_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
# file_pattern = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_*_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{epochs:03d}_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
file_pattern = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_ep_*.pt"
print(file_pattern)

In [ ]:
# filelist = glob.glob(os.path.join('ckpts',file_pattern))
filelist = glob.glob(file_pattern,root_dir = './ckpts')
filelist = sorted(filelist)
filelist
epochlist =sorted([int(x[-6:-3]) for x in filelist])
epochlist
# epochlist =[int(x[31:34]) for x in filelist]
# epochlist =[int(x[26:29]) for x in filelist]
# epochlist = [200]
# epochlist = [350]

In [ ]:
# = utils.load_model_from_ckpt(model, ex_mode, ex_timestamp, ex_title, epochs, ex_cpb, ex_factor )
# mdl_outputs = utils.run_model_on_test_data(model, test_loader, embedding_layer = 4)
metric = 'cosine'
# metric = 'euclidian'
# metric = 'correlation'
df_metrics = dict()
grp_level_dict = {}
for ds in ['train', 'val', 'test']:
    df_metrics[ds] = pd.DataFrame()
    grp_level_dict[ds] = {}

In [ ]:
for epochs in epochlist :
    # for filename in filelist:
    filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_ep_{epochs:03d}.pt"
    model = utils.load_model_from_ckpt(model, filename = filename, v1=True, verbose = False)
    if model is None:
        print(f"model is none")
        break
    
    for ds in ['train', 'val', 'test']:        
        mdl_outputs = utils.run_model_on_test_data(model, data_loader[ds], embedding_layer = 4, verbose = False)
        mdl_outputs.epochs = epochs
        reg_metrics = utils.compute_regression_metrics(mdl_outputs)
        sample_activations = utils.distance_metrics_sample_set(mdl_outputs, num_samples = 10, display = False)
        dist_metrics, grp_level_dict[ds][epochs]  = utils.compute_distance_metrics(sample_activations, epochs = epochs, metric = metric,display = False )    
        # utils.display_dist_metrics(dist_metrics, epochs, metric)
        # utils.display_regr_metrics(reg_metrics)\
        try:
            df_metrics[ds] = pd.concat((df_metrics[ds], pd.DataFrame(vars(reg_metrics) | dist_metrics , index = [epochs] )) , verify_integrity=True)
        except ValueError:
            print(f" {epochs} already exists - not added")

In [ ]:
df_metrics.keys()
mdl_outputs.latent_embedding.shape
grp_level_dict.keys()

In [ ]:
def distance_matrices(grp_level_dict, epochs = [50,150,250,350]):
    import seaborn as sns
    # plotting the heatmap 
    # del fig, axs
    len_dict = len(grp_level_dict)
    ## figsize =(width, height)
    cols = 3
    rows = len(epochs)
    fig, axs = plt.subplots(rows,cols, sharey=False, tight_layout=True, figsize=(cols*5,rows*4) )
     
    
    row_id = 0
    col_id = 0
    for k in epochs:
        hm = sns.heatmap(data=grp_level_dict['test'][k]['inp'][0],annot=True, ax = axs[row_id,0]) 
        hm = sns.heatmap(data=grp_level_dict['test'][k]['emb'][0],annot=True, ax = axs[row_id,1]) 
        hm = sns.heatmap(data=grp_level_dict['test'][k]['out'][0],annot=True, ax = axs[row_id,2]) 
        # if col_id == 3:
        #     row_id += 1
        #     col_id = 0 
        # else:
            # col_id += 1
        row_id += 1
    # displaying the plotted heatmap 
    plt.show()

# Display results dataframe

In [ ]:
metric_cols =  df_metrics['train'].columns.to_list()
for key in df_metrics.keys():
    df_metrics[key].sort_values('epochs', inplace = True)
    # df_metrics[key]
# metric_cols


pattern1 = re.compile(r"(inp|emb|out)_(all|same|diff)_(avg)")
pattern2 = re.compile(r"(CL)_(inp|emb|out)_(all|same|diff)_(avg)")
cols0 = ['epochs', 'R2_score', 'rmse_score', 'mse_score','mae_score']
cols0 = ['epochs', 'R2_score', 'rmse_score']
cols1 = ['epochs'] +[ col for col in metric_cols if pattern1.match(col) ]
cols2 = ['epochs'] +[ col for col in metric_cols if pattern2.match(col) ]


In [ ]:
# df_metrics['train'][cols0]
# df_metrics['val'][cols0]
# df_metrics['test'][cols0]

### Autoencoder with SNNL regularization , embedding 50 dim


In [ ]:
df_tmp = pd.merge(df_metrics['train'][cols0],df_metrics['test'][cols0], how='inner', left_on = 'epochs', right_on = 'epochs', suffixes=('_train', None), indicator = False)
df_merge_metrics = pd.merge(df_tmp, df_metrics['val'][cols0], how='inner', left_on = 'epochs', right_on = 'epochs', copy = True, suffixes=('_test', '_val'), indicator = False)
df_merge_metrics[['epochs', 'R2_score_train','R2_score_val','R2_score_test','rmse_score_train','rmse_score_val','rmse_score_test']]

In [ ]:
df_metrics['train'].plot(x = 'epochs', y=['R2_score','rmse_score'], kind='line', subplots=False)
df_metrics['test'].plot(x = 'epochs', y=['R2_score','rmse_score'], kind='line', subplots=False)
df_merge_metrics.plot(x = 'epochs', y=['R2_score_train','R2_score_val','R2_score_test',], kind='line', subplots=False, title = 'R2 score for AE with 50 dim embedding layer')
# I computed the R2 against the training and test data on the model that uses the SNN regularization, they are very similar (for example training = 0.3580, test= 0.3636) - this is for the latent layer size of 100. The
# Now I am running models without any regularization to with different latent layer sizes to see how the latent layer size affects the R2

### Autoencoder with SNNL regularization , embedding 100 dim


In [ ]:
df_tmp = pd.merge(df_metrics['train'][cols0],df_metrics['test'][cols0], how='inner', left_on = 'epochs', right_on = 'epochs', suffixes=('_train', None), indicator = False)
df_merge_metrics = pd.merge(df_tmp, df_metrics['val'][cols0], how='inner', left_on = 'epochs', right_on = 'epochs', copy = True, suffixes=('_test', '_val'), indicator = False)
df_merge_metrics[['epochs', 'R2_score_train','R2_score_val','R2_score_test','rmse_score_train','rmse_score_val','rmse_score_test']]

In [ ]:
df_merge_metrics.plot(x = 'epochs', y=['R2_score_train','R2_score_val','R2_score_test',], kind='line', subplots=False, title = 'R2 score for AE with 100 dim embedding layer')

### Baseline autoencoder , embedding 150 dim


In [ ]:
df_tmp = pd.merge(df_metrics['train'][cols0],df_metrics['test'][cols0], how='inner', left_on = 'epochs', right_on = 'epochs', suffixes=('_train', None), indicator = False)
df_merge_metrics = pd.merge(df_tmp, df_metrics['val'][cols0], how='inner', left_on = 'epochs', right_on = 'epochs', copy = True, suffixes=('_test', '_val'), indicator = False)
df_merge_metrics[['epochs', 'R2_score_train','R2_score_val','R2_score_test','rmse_score_train','rmse_score_val','rmse_score_test']]
df_merge_metrics.plot(x = 'epochs', y=['R2_score_train','R2_score_val','R2_score_test',], kind='line', subplots=False, title = 'R2 score for AE with 150 dim embedding layer')

In [ ]:
distance_matrices(grp_level_dict, epochs = [50,100,200,300])

### Baseline autoencoder , embedding 50 dim

In [ ]:
df_tmp = pd.merge(df_metrics['train'][cols0],df_metrics['test'][cols0], how='inner', left_on = 'epochs', right_on = 'epochs', suffixes=('_train', None), indicator = False)
df_merge_metrics = pd.merge(df_tmp, df_metrics['val'][cols0], how='inner', left_on = 'epochs', right_on = 'epochs', copy = True, suffixes=('_test', '_val'), indicator = False)
df_merge_metrics[['epochs', 'R2_score_train','R2_score_val','R2_score_test','rmse_score_train','rmse_score_val','rmse_score_test']]
df_merge_metrics.plot(x = 'epochs', y=['R2_score_train','R2_score_val','R2_score_test',], kind='line', subplots=False, title = 'R2 score for AE with 50 dim embedding layer')

In [ ]:
distance_matrices(grp_level_dict, epochs = [50,100,150,200])

## Write to csv and Excel files

In [ ]:
timestamp = datetime.now().strftime('%m%d_%H%M%S')
csv_filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{metric}_metrics_{timestamp}.csv"
csv_filename = os.path.join('./results', csv_filename)
csv_filename

excel_filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{metric}_metrics_{timestamp}.xlsx"
excel_filename = os.path.join('./results', excel_filename)
excel_filename

In [ ]:
excel_sheet = f"{'bl' if ex_runmode == 'baseline' else ex_runmode}_{timestamp}"
excel_sheet

In [ ]:
df_metrics.to_csv(csv_filename, index = False)
df_metrics.to_excel(excel_filename, sheet_name=excel_sheet, index_label='id')

In [ ]:
# mdl_outputs.embeddings[9].sum()
# mdl_outputs.y_pred.sum()
# mdl_outputs.compounds[:17]
df_metrics

## Read metrics CSV file

In [ ]:
input_file  = './results/AE_snnl_20240516_snglOpt-100Ltnt_euclidean_metrics_0603_130545.csv'
input_metrics = pd.read_csv(input_file)
input_metrics

In [ ]:
pattern1 = re.compile(r"(inp|emb|out)_(all|same|diff)_(avg)")
pattern2 = re.compile(r"(CL)_(inp|emb|out)_(all|same|diff)_(avg)")
cols1 = ['epochs', 'R2_score', 'rmse_score'] + [ col for col in metric_cols if pattern1.match(col) ]
cols2 = ['epochs', 'R2_score', 'rmse_score'] +[ col for col in metric_cols if pattern2.match(col) ]

In [ ]:
input_metrics[cols1]

# Metrics

## Distance Metrics

In [ ]:
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=250, nanstr='nan')

In [ ]:
# 1189.5788218820562 + 16009.773130260974 
# 2175.389955100794 / 50

In [ ]:
dmtrc = compute_distance_metrics(sample_activations, epochs, metric = 'euclid')

In [ ]:
# dmtrc.keys()

In [ ]:
dmtrc = compute_distance_metrics(sample_activations, epochs, metric = 'euclid')

In [ ]:
dmtrcs = compute_distance_metrics(sample_activations, epochs)

In [ ]:
display_distance_metrics(dmtrc, epochs)

In [ ]:
# print(f'INPUT features - Euclidean distances - {epochs} epochs')
# print('-'*20)
# dmtrc = distance_metric_1(inp_euc_distance)

# print(f'EMBEDDED features - Euclidean distances - {epochs} epochs')
# print('-'*20)
# dmtrc = distance_metric_1(emb_euc_distance)

# print(f'RECONSTURCTED features -  Euclidean distances - {epochs} epochs')
# print('-'*20)
# dmtrc = distance_metric_1(out_euc_distance)

### Cosine Distance 

In [ ]:
print(f'Input features - Cosine - {epochs} epochs')
print('-'*20)
grp_level_input = distance_metric_1(inp_cos_distance)
 
print(f'Output features - Cosine - {epochs} epochs')
print('-'*20)
grp_level_output = distance_metric_1(out_cos_distance)

In [ ]:
# inp_cos_distance[:5,:15]
# out_cos_distance[:5,:15]
# emb_cos_distance[:5,:15]

## Classification Metrics 

In [ ]:
cls_metrics = compute_classification_metrics(mdl_output)
cls_metrics.epochs = epochs
display_classification_metrics(cls_metrics)
plot_classification_metrics(cls_metrics)

#### 

In [ ]:
# test_accuracy = binary_accuracy(y_true=labels, y_prob=logits)
# test_f1 = binary_f1_score(y_true=labels, y_prob=logits)
# print(f"accuracy: {test_accuracy}%      F1 Score: {test_f1:.4f}")
# cm = confusion_matrix(y_true = labels, y_pred = (logits >= 0.5))
# disp = ConfusionMatrixDisplay(cm)
# disp.plot()


In [ ]:
# filename = f"DNN-{args.model.lower()}-{args.seed}.json"
# export_results(model=model, filename=filename)

In [ ]:
# for i in range(len(logits)):
#     if labels[i] == 1:
#         print(f" y_pred: {logits[i]:.5e}   {logits[i]>=0.500}  \t  y_true: {labels[i]}")

## Regression Metrics 

In [ ]:
mdl_outputs.epochs = epochs
reg_metrics = compute_regression_metrics(mdl_outputs)
# reg_metrics
print()
display_regr_metrics(reg_metrics)
df_metrics

In [ ]:
try:
    df_metrics = pd.concat((df_metrics, pd.DataFrame(vars(reg_metrics), index = [epochs] )) , verify_integrity=True)
    df_metrics.sort_index(inplace = True)
except ValueError:
    print(f" {epochs} already exists - not added")

In [ ]:
df_metrics

## Results

  #### Results `20240505_autoencoder`

In [ ]:
 metrics as epoch 25     metrics as epoch 50     metrics as epoch 100   metrics as epoch 125   metrics as epoch 150  
---------------------- -----------------------  ---------------------- ---------------------- ---------------------- 
RMSE Score :  0.543655  RMSE Score :  0.539906   RMSE Score : 0.536459  RMSE Score : 0.624126  RMSE Score : 0.624126 
 MSE Score :  0.317944   MSE Score :  0.314051    MSE Score : 0.310318   MSE Score : 0.408070   MSE Score : 0.408070 
 MAE Score :  0.378762   MAE Score :  0.373654    MAE Score : 0.371468   MAE Score : 0.434099   MAE Score : 0.434099 
  R2 Score :  0.611225    R2 Score :  0.615898     R2 Score : 0.632036    R2 Score : 0.518840    R2 Score : 0.518840 

  #### Results ` baseline 20240505_dropout04`

In [ ]:
 metrics at epoch  25   metrics at epoch  50   metrics at epoch  75   metrics at epoch 100   metrics at epoch 125   metrics at epoch 150   metrics at epoch 175   metrics at epoch 200   metrics at epoch 225   metrics at epoch 250   metrics at epoch 300   
---------------------- ---------------------- ---------------------- ---------------------- ---------------------- ---------------------- ---------------------- ---------------------- ---------------------- ---------------------- ----------------------  
RMSE Score :  0.539952 RMSE Score :  0.522212 RMSE Score :  0.516986 RMSE Score :  0.510535 RMSE Score :  0.512396 RMSE Score :  0.507615 RMSE Score :  0.501631 RMSE Score :  0.504048 RMSE Score :  0.505369 RMSE Score :  0.504147 RMSE Score :  0.503990  
 MSE Score :  0.313584  MSE Score :  0.295068  MSE Score :  0.289945  MSE Score :  0.283578  MSE Score :  0.285415  MSE Score :  0.280749  MSE Score :  0.275116  MSE Score :  0.277431  MSE Score :  0.278716  MSE Score :  0.277734  MSE Score :  0.277629  
 MAE Score :  0.372594  MAE Score :  0.361990  MAE Score :  0.357078  MAE Score :  0.353107  MAE Score :  0.353546  MAE Score :  0.350261  MAE Score :  0.345679  MAE Score :  0.346366  MAE Score :  0.347385  MAE Score :  0.346365  MAE Score :  0.345845  
  R2 Score :  0.610749   R2 Score :  0.630615   R2 Score :  0.648668   R2 Score :  0.661354   R2 Score :  0.658380   R2 Score :  0.622958   R2 Score :  0.670584   R2 Score :  0.674873   R2 Score :  0.670612   R2 Score :  0.670147   R2 Score :  0.670171   

  #### Results `snnl 20240507_secondTry`

In [ ]:
  metrics as epoch  25   metrics as epoch  50    metrics as epoch  75   metrics as epoch 100    metrics as epoch 125    metrics at epoch 150   metrics at epoch 175   metrics at epoch 200         
----------------------  ----------------------  ---------------------- ----------------------  ----------------------  ---------------------- ---------------------- ----------------------       
RMSE Score : 0.550294   RMSE Score :  0.536259  RMSE Score :  0.516979 RMSE Score :  0.513265  RMSE Score :  0.511261  RMSE Score :  0.506228 RMSE Score :  0.508196 RMSE Score :  0.508642       
 MSE Score : 0.324991    MSE Score :  0.310259   MSE Score :  0.290345  MSE Score :  0.286497   MSE Score :  0.284502   MSE Score :  0.279736  MSE Score :  0.281590  MSE Score :  0.282127       
 MAE Score : 0.382500    MAE Score :  0.372185   MAE Score :  0.358323  MAE Score :  0.353431   MAE Score :  0.351594   MAE Score :  0.347926  MAE Score :  0.348469  MAE Score :  0.348470       
  R2 Score : 0.588431     R2 Score :  0.625873    R2 Score :  0.652229   R2 Score :  0.657811    R2 Score :  0.656971    R2 Score :  0.667756   R2 Score :  0.665126   R2 Score :  0.661970        

### metrics per epoc

#### 25 epochs

In [ ]:
display_regr_metrics(reg_metrics)

#### 50 epochs

In [ ]:
display_regr_metrics(reg_metrics)

#### 75 epochs

In [ ]:
display_regr_metrics(reg_metrics)

#### 100 epochs

In [ ]:
display_regr_metrics(reg_metrics)

#### 125 epochs

In [ ]:
display_regr_metrics(reg_metrics)

#### 150 epochs

In [ ]:
display_regr_metrics(reg_metrics)

#### 175 epochs

In [ ]:
display_regr_metrics(epochs, rm)

#### 200 epochs

In [ ]:
display_regr_metrics(reg_metrics)

## Apply TSNe

In [ ]:
tsne = TSNE(2)
projection = dict()
num_layers = len(mdl_outputs.embeddings)
for k in range(num_layers):
    projection[k] = tsne.fit_transform(mdl_outputs.embeddings[k])
    print(f" layer {k+1} of {num_layers}: projection shape: {projection[k].shape}")
    

In [ ]:
# projection_subset = {x: projection[x] for x in range(6)}

# Plot TSNE embeddings

### 100 Epochs  

In [ ]:
# plot_TSNE(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', end = 21, epoch = mdl_outputs.epochs)
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', items = list(range(21)), layers = [0,1,2,3,4], epoch = mdl_outputs.epochs)
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', items = list(range(21)), layers = [5,6,7,8,9], epoch = mdl_outputs.epochs)
print('\n')
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', layers = [0,8,9],  items = list(range(9)), epoch = mdl_outputs.epochs, limits=(-50,50))
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', layers = [0,8,9],  items = [3,4,5], epoch = mdl_outputs.epochs, limits=(-50,50))
print('\n')
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='tpsa', layers = [0,1,2,3,4],epoch = mdl_outputs.epochs)
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='tpsa', layers = [5,6,7,8,9],epoch = mdl_outputs.epochs)

### 200 Epochs  

In [ ]:
# plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', items = list(range(21)), layers = [0,1,2,3,4], epoch = mdl_outputs.epochs)
# plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', items = list(range(21)), layers = [5,6,7,8,9], epoch = mdl_outputs.epochs)
# print('\n')
# plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', layers = [0,8,9],  items = list(range(9)), epoch = mdl_outputs.epochs, limits=(-50,50))
# plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', layers = [0,8,9],  items = [3,4,5], epoch = mdl_outputs.epochs, limits=(-50,50))

print('\n')
fig1 = plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='tpsa', layers = [0,1,2,3,4],epoch = mdl_outputs.epochs)
fig2 = plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='tpsa', layers = [5,6,7,8,9],epoch = mdl_outputs.epochs)

In [ ]:
filename1 = f"./images/{model.name}_{ex_mode}_{timestamp}_{title}_{epochs:03d}_allcmpds_tpsa_1.jpg"
filename2 = f"./images/{model.name}_{ex_mode}_{timestamp}_{title}_{epochs:03d}_allcmpds_tpsa_2.jpg"
fig1.savefig(filename1)
fig2.savefig(filename2)

In [ ]:
Image('test.png')

### 225 Epochs  

In [ ]:
# plot_TSNE(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', end = 21, epoch = mdl_outputs.epochs)
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', items = list(range(21)), layers = [0,1,2,3,4], epoch = mdl_outputs.epochs)
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', items = list(range(21)), layers = [5,6,7,8,9], epoch = mdl_outputs.epochs)

In [ ]:
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', layers = [0,4,8,9],  items = list(range(9)), epoch = mdl_outputs.epochs, limits=(-50,50))
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', layers = [0,4,8,9],  items = [3,4,5], epoch = mdl_outputs.epochs, limits=(-50,50))

In [ ]:
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='tpsa', layers = [0,1,2,3,4],epoch = mdl_outputs.epochs)
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='tpsa', layers = [5,6,7,8,9],epoch = mdl_outputs.epochs)

### 300 Epochs  

In [ ]:
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='tpsa', layers = [1,3,4,6,8,9],epoch = mdl_outputs.epochs)

In [ ]:
plot_TSNE_2(projection, mdl_outputs.labels, mdl_outputs.comp_labels, key='compound', layers = [0,4,9],  items = list(range(9)), epoch = mdl_outputs.epochs)

# Older plots

### Based on Compound 

In [ ]:
for layer in [0,1,2,3,4]:
    end = 15
    df = pd.DataFrame(dict(
        x=projection[layer][:end,0],
        y=projection[layer][:end,1],
        label=labels[0:end],
        comp_label = comp_labels[0:end]
    ))
    # _=plt.figure()
    lp=sb.lmplot( data=df, x ="x", y = "y", hue='comp_label', fit_reg=False, palette="hls") #, size=6)
    _=lp.fig.suptitle(f'layer {layer}')

plt.show()

### Based on TPSA Indicator 

In [ ]:
end = 1200
for layer in [0,1,2,3,4]:
    df = pd.DataFrame(dict(
        x=projection[layer][:end,0],
        y=projection[layer][:end,1],
        label=labels[0:end]
    ))
    lp= sb.lmplot( data=df, x ="x", y = "y", hue='label', fit_reg=False, palette="hls") #, size=6)
    _=lp.fig.suptitle(f'layer {layer}')
plt.show()

### 3D ? 

In [ ]:
from matplotlib import cm
x=projection[4][:,0]
y=projection[4][:,1]
# z=projection[4][:,2]
label=np.expand_dims(labels,-1)
z = np.expand_dims(z,-1)
z.shape

ax = plt.figure(figsize=(8,8)).add_subplot(projection='3d')
# Plot the 3D surface
# ax.plot_surface(x, y, z, edgecolor='royalblue', lw=0.5, rstride=8, cstride=8, alpha=0.3)
# ax.plot_trisurf(x, y, z, linewidth=0 , )
# surf = ax.plot_surface(x, y, label, cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax.scatter(x, y, label)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()
ax.view_init(elev=45.)

plt.show()

# PCA and TSNE

## define PCA model

In [ ]:
loader_iter = iter(train_loader)

In [ ]:
(batch_features, batch_labels, batch_plates, batch_compounds, batch_hash) = next(loader_iter)

In [ ]:
type(batch_features), batch_features.shape, batch_labels.numpy().shape

In [ ]:
pca = PCA(n_components=200, copy=False, whiten=False, 
          svd_solver='auto', tol=0.0, iterated_power='auto', 
          n_oversamples=10, power_iteration_normalizer='auto', random_state=None)

In [ ]:
pca.fit(batch_features)
X = pca.transform(batch_features)

## define TSNE 

In [ ]:
tsne = TSNE()
projection = tsne.fit_transform(X)
projection.shape

In [ ]:
rows = 1000
df = pd.DataFrame(dict(
    x=projection[:rows,0],
    y=projection[:rows,1],
    label=batch_labels.numpy()[:rows]
    # moa=pd.concat([treated_wells["MOA"], treatments["MOA"]]),
    # batch=pd.concat([treated_wells["Plate"].apply(lambda x: x.split("_")[0]), treatments["Batch"]]),
    # compound=pd.concat([treated_wells["Compound"], treatments["Compound"]]),
    # correct=pd.concat([treated_wells["nsc"],treatments["nsc"]]),
    # level=pd.concat([treated_wells["level"],treatments["level"]])
))

df.info()
df.head()

In [ ]:
plt.figure()
# sb.lmplot('x', 'y', data=df, hue='label', fit_reg=False, palette="hls", size=6)
sb.lmplot( data=df, x ="x", y = "y", hue='label', fit_reg=False, palette="hls") #, size=6)


## Apply TSNE on embeddings

In [ ]:
tsne = TSNE(2)
projection = dict()
for k,v in embeddings.items():
    projection[k] = tsne.fit_transform(v)
    projection[k].shape

#### Layer 0 

In [ ]:
df = pd.DataFrame(dict(
    x=projection[0][:,0],
    y=projection[0][:,1],
    label=labels
))
plt.figure()
sb.lmplot( data=df, x ="x", y = "y", hue='label', fit_reg=False, palette="hls") #, size=6)

#### Layer 1 

In [ ]:
df = pd.DataFrame(dict(
    x=projection[1][:,0],
    y=projection[1][:,1],
    label=labels
))
plt.figure()
sb.lmplot( data=df, x ="x", y = "y",hue='label', fit_reg=False, palette="hls") #, size=6)

#### Layer 2 

In [ ]:
df = pd.DataFrame(dict(
    x=projection[2][:,0],
    y=projection[2][:,1],
    label=labels
))
plt.figure()
sb.lmplot( data=df, x ="x", y = "y",hue='label', fit_reg=False, palette="hls") #, size=6)

#### Layer 3 

In [ ]:
df = pd.DataFrame(dict(
    x=projection[3][:,0],
    y=projection[3][:,1],
    label=labels
))
plt.figure()
sb.lmplot( data=df, x ="x", y = "y",hue='label', fit_reg=False, palette="hls") #, size=6)

#### Layer 4 

In [ ]:
df = pd.DataFrame(dict(
    x=projection[4][:,0],
    y=projection[4][:,1],
    label=labels
))
plt.figure()
sb.lmplot( data=df, x ="x", y = "y",hue='label', fit_reg=False, palette="hls") #, size=6)

In [ ]:
np.bincount(labels.astype(np.int64))

In [ ]:
from matplotlib import cm
x=projection[4][:,0]
y=projection[4][:,1]
# z=projection[4][:,2]
label=np.expand_dims(labels,-1)
z = np.expand_dims(z,-1)
z.shape

ax = plt.figure(figsize=(8,8)).add_subplot(projection='3d')
# Plot the 3D surface
# ax.plot_surface(x, y, z, edgecolor='royalblue', lw=0.5, rstride=8, cstride=8, alpha=0.3)
# ax.plot_trisurf(x, y, z, linewidth=0 , )
# surf = ax.plot_surface(x, y, label, cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax.scatter(x, y, label)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()
ax.view_init(elev=45.)

plt.show()